SIDE NOTE: would love to get any critiques, hints, tips, oberservations on any level not just on ML (but obviously ML as the priority).  Beginner Python learner (though I took the Full Stack Course) so any suggestions there are welcome.  Thanks.

Will need to import these libs plus setting some global variables

In [1]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled', 'games_played', 'round']
sub_cols = ['current_formation', 'avg_goals_against', 'goal_diff', 'win_percentage', 'sos',
           'opp_win_percentage', 'opp_sos', 'current_team_yellow_cards', 'current_team_corner_kicks', 'current_team_first_half_goals', 'current_team_sec_half_goals', 
           'opp_team_yellow_cards', 'opp_team_corner_kicks', 'opp_team_first_half_goals', 'opp_team_sec_half_goals']

all_models = ['log', 'svc', 'gmm', 'knn', 'gnb', 'randomForest']

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = True

INITIALIZED...


In [2]:
data_csv = 'raw_data_squared.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)
print('Data Loaded...')
print("Dataset size :: {}".format(raw_data.shape))
display(raw_data.head())

ROUND 4 :: TEAM ID 21
ROUND 5 :: TEAM ID 21
ROUND 6 :: TEAM ID 21
ROUND 7 :: TEAM ID 21
ROUND 8 :: TEAM ID 21
ROUND 9 :: TEAM ID 21
ROUND 10 :: TEAM ID 21
ROUND 11 :: TEAM ID 21
ROUND 12 :: TEAM ID 21
ROUND 13 :: TEAM ID 21
ROUND 14 :: TEAM ID 21


/Users/senzari/.local/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


ROUND 15 :: TEAM ID 21
ROUND 16 :: TEAM ID 21
ROUND 18 :: TEAM ID 21
ROUND 19 :: TEAM ID 21
ROUND 21 :: TEAM ID 21
ROUND 23 :: TEAM ID 21
ROUND 24 :: TEAM ID 21
ROUND 25 :: TEAM ID 21
ROUND 26 :: TEAM ID 21
ROUND 27 :: TEAM ID 21
ROUND 5 :: TEAM ID 22
ROUND 6 :: TEAM ID 22
ROUND 7 :: TEAM ID 22
ROUND 8 :: TEAM ID 22
ROUND 9 :: TEAM ID 22
ROUND 10 :: TEAM ID 22
ROUND 11 :: TEAM ID 22
ROUND 12 :: TEAM ID 22
ROUND 13 :: TEAM ID 22
ROUND 15 :: TEAM ID 22
ROUND 16 :: TEAM ID 22
ROUND 18 :: TEAM ID 22
ROUND 19 :: TEAM ID 22
ROUND 21 :: TEAM ID 22
ROUND 22 :: TEAM ID 22
ROUND 23 :: TEAM ID 22
ROUND 24 :: TEAM ID 22
ROUND 25 :: TEAM ID 22
ROUND 26 :: TEAM ID 22
ROUND 27 :: TEAM ID 22
ROUND 5 :: TEAM ID 23
ROUND 6 :: TEAM ID 23
ROUND 7 :: TEAM ID 23
ROUND 8 :: TEAM ID 23
ROUND 9 :: TEAM ID 23
ROUND 10 :: TEAM ID 23
ROUND 11 :: TEAM ID 23
ROUND 12 :: TEAM ID 23
ROUND 13 :: TEAM ID 23
ROUND 14 :: TEAM ID 23
ROUND 15 :: TEAM ID 23
ROUND 16 :: TEAM ID 23
ROUND 17 :: TEAM ID 23
ROUND 18 :: TEAM ID 2

,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,goals_for,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks,goals,points
0,249,21,FC Dallas,33,DC United,2016-03-26 21:30:00,4,3,0,4-2-3-1,4,5,-120.111111,-62.111111,-6997.333333,-7293.000000,3396.000000,722.333333,3,3
1,255,21,FC Dallas,39,Columbus Crew,2016-04-03 01:00:00,5,3,1,4-2-3-1,5,4,-88.777778,-37.777778,-9370.555556,-4396.333333,3582.222222,-1911.555556,1,1
2,265,21,FC Dallas,32,San Jose Earthquakes,2016-04-10 01:00:00,6,3,1,4-4-2,6,5,-189.000000,-50.111111,-9633.000000,-9976.000000,3680.555556,1786.666667,2,1
3,272,21,FC Dallas,30,Portland Timbers,2016-04-14 02:30:00,7,3,0,4-4-2,6,7,-150.777778,-57.777778,-8341.666667,-7280.000000,2577.777778,4138.666667,3,3
4,282,21,FC Dallas,27,Sporting Kansas City,2016-04-17 23:00:00,7,3,1,4-4-2,6,2,-150.777778,-165.000000,-8341.666667,-10975.888889,2577.777778,1530.555556,2,3


## FORMATTING

Converting Goals to our binary classification (0-1) and (2+) and also converting the sos and rpi and implementing the team to a particular rpi quartile

In [3]:
# Helper Function - Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

""" Need to do some formatting of the Data before we run the models"""
pd.set_option('display.max_columns', 500)

""" Formatting data to convert goals scored to the correct category"""
# Not using points as a target for this version, using goals
classifier_data = raw_data.drop('points', 1)

classifier_data['converted_goals'] = classifier_data.apply(lambda row: model_libs.set_group(row['goals']), axis=1)

classifier_data = classifier_data.drop(ignore_cols + ['current_formation', 'goals'], 1)
display(classifier_data.head())

,is_home,goals_for,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks,converted_goals
0,0,4,5,-120.111111,-62.111111,-6997.333333,-7293.000000,3396.000000,722.333333,1
1,1,5,4,-88.777778,-37.777778,-9370.555556,-4396.333333,3582.222222,-1911.555556,0
2,1,6,5,-189.000000,-50.111111,-9633.000000,-9976.000000,3680.555556,1786.666667,1
3,0,6,7,-150.777778,-57.777778,-8341.666667,-7280.000000,2577.777778,4138.666667,1
4,1,6,2,-150.777778,-165.000000,-8341.666667,-10975.888889,2577.777778,1530.555556,1


### RUNNING CLASSIFICATION MODEL ON IF TEAMS SCORE 0-1 OR 1-2 ON GAMES

In [4]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    
    #display(new_data.head())
    
    (y, X) = model_libs._extract_target(new_data, target)
    
    models = form_model.train_models(round_number, X, y, models)
    
    return models


#### Running ALL Features 
models_test_1 = run_features(classifier_data, [], 'converted_goals', ["knn"])

(classifier_y, classifier_X) = model_libs._extract_target(classifier_data, 'converted_goals')

def check_accuracy(model, data_X):
    actual_y = pd.DataFrame(classifier_y.values, columns=['actual'])
    predictions = pd.DataFrame(model.predict(data_X), columns=['predictions'])
    preds = pd.concat([predictions, actual_y], axis=1)
    preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
    accuracy = np.divide(preds['diff'].sum(), float(len(preds['diff'])))
    print(accuracy)

for m in models_test_1:
    check_accuracy(m, classifier_X)

-----------------------------------
Training K Neighbors Classifier Model
KNN Score on Training Set :: 0.76134122288
KNN Score on Test Set:: 0.590551181102
Finished K-Means Modeling
0.727129337539


In [ ]:
def train_tuned_models(round_num, X, y):
    for i in all_models:
        models = form_model.build_tuned_model(X, y, i)
        
    return models
        
tuned_models = train_tuned_models(round_number, classifier_X, classifier_y)

In [5]:
random_forest_model = form_model.build_tuned_model(classifier_X, classifier_y, 'randomForest')

for m in random_forest_model:
    print(m)
    check_accuracy(m, classifier_X)

-----------------------------------
Training Random Forest Model
[ 0.59803922  0.54901961  0.58415842  0.56435644  0.57425743]
Accuracy: 0.57 (+/- 0.03)
Finished Random Forest Modeling
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=2, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.714511041009


In [6]:
knn_model = form_model.build_tuned_model(classifier_X, classifier_y, 'knn')

for m in knn_model:
    check_accuracy(m, classifier_X)

-----------------------------------
Training K-Means Model
KNN Score :: 0.747534516765 for Training
KNN Score :: 0.590551181102 for Testing
KNN Score :: 0.775147928994 for Training
KNN Score :: 0.543307086614 for Testing
KNN Score :: 0.769230769231 for Training
KNN Score :: 0.574803149606 for Testing
KNN Score :: 0.77909270217 for Training
KNN Score :: 0.622047244094 for Testing
KNN Score :: 0.781496062992 for Training
KNN Score :: 0.563492063492 for Testing
Finished K-Means Modeling
0.738170347003


In [7]:
gnb_model = form_model.build_tuned_model(classifier_X, classifier_y, 'gnb')

for m in gnb_model:
    print(m)
    check_accuracy(m, classifier_X)

-----------------------------------
Training Gaussian NB Model
[ 0.59803922  0.54901961  0.53465347  0.6039604   0.62376238]
Accuracy: 0.58 (+/- 0.07)
Finished Gaussian NB Modeling
GaussianNB()
0.618296529968


In [8]:
svc_model = form_model.build_tuned_model(classifier_X, classifier_y, 'svc')

for m in svc_model:
    print(m)
    check_accuracy(m, classifier_X)

Training and Tuning SVC Model
[ 0.58661417  0.58498024]
Accuracy: 0.59 (+/- 0.00)
Finished SVC Modeling
GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 10, 100]}, {'kernel': ['linear'], 'C': [1, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)
0.929022082019


In [9]:
gmm_model = form_model.build_tuned_model(classifier_X, classifier_y, 'gmm')

for m in gmm_model:
    print(m)
    check_accuracy(m, classifier_X)

-----------------------------------
Training and Tuning GMM Model
W/ Covariance Type :: spherical
# of Components :: 2
Silhouette Score :: 0.277676503481 for Training
Silhouette Score :: 0.264571641401 for Testing
# of Components :: 3
Silhouette Score :: 0.298573567568 for Training
Silhouette Score :: 0.285565714885 for Testing
# of Components :: 4
Silhouette Score :: 0.205464726364 for Training
Silhouette Score :: 0.145144236591 for Testing
W/ Covariance Type :: tied
# of Components :: 2
Silhouette Score :: 0.335626491871 for Training
Silhouette Score :: 0.297992036355 for Testing
# of Components :: 3
Silhouette Score :: 0.300929798319 for Training
Silhouette Score :: 0.209417787854 for Testing
# of Components :: 4
Silhouette Score :: 0.11072761526 for Training
Silhouette Score :: 0.0900983866386 for Testing
W/ Covariance Type :: diag
# of Components :: 2
Silhouette Score :: 0.253705239525 for Training
Silhouette Score :: 0.257196149323 for Testing
# of Components :: 3
Silhouette Scor

# Optimize Features

In [ ]:
def plot_RFECV(X, y):
    svc = SVC(kernel="linear")
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y, 2),
                  scoring='accuracy')
    rfecv.fit(X, y)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()
    
plot_RFECV(classifier_X, classifier_y)

In [ ]:
def create_RFE(X, y):
    # Create the RFE object and rank each pixel
    svc = SVC(kernel="linear", C=1)
    rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
    rfe.fit(X, y)
    print(rfe.ranking_)
    return rfe.ranking_

rankings = create_RFE(classifier_X, classifier_y)

In [ ]:
""" Reordering the columns in the data to reflect the rankings """
def rank_columns(data_X, rankings):
    rankings_name = []
    ranked_X = data_X

    for r in range(len(rankings)):
        column_name = data_X.iloc[:, rankings[r]-1].name
        rankings_name.append(column_name)

    ranked_X = ranked_X.reindex_axis(rankings_name, axis=1)
    
    return ranked_X

ranked_X = rank_columns(classifier_X, rankings)
display(ranked_X.head())
display(ranked_X.columns)

In [ ]:
def train_ranked_models(X, y):
    
    for x in xrange(15, 18):
        print('X :: {}'.format(x))

        ranked_models = form_model.train_models(round_number, X.iloc[:, 1:x], y, ['log',  'svc', 'gmm', 'knn', 'gnb', 'randomForest'])
        
train_ranked_models(ranked_X, classifier_y)

In [ ]:
def plot_PCA(data):
    
    pca = PCA(n_components=4)
    pca.fit(data)
    
    # Generate PCA results plot
    pca_results = rs.pca_results(data, pca)
    #print(pca_results)
    pca_data = pca.transform(data)
    
    # Create a DataFrame for the reduced data
    pca_data = pd.DataFrame(pca_data, columns = ['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4'])
    #plot_data = pd.concat([preds, pca_data], axis = 1)
    
    # Scatterplotting the transformed data if it's 2 Dimensions
    """fig, ax = plt.subplots(figsize = (14,8))
    cmap = cm.get_cmap('gist_rainbow')
    
    for i, cluster in plot_data.groupby('converted_goals'):   
        cluster.plot(ax = ax, kind = 'scatter', x = 'Dimension 1', y = 'Dimension 2', 
                     color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);"""
    
    return pca, pca_data, pca_results

pca, pca_data, pca_results = plot_PCA(classifier_X)

all_pca_data = pd.concat([classifier_y, pca_data], axis = 1)

print(pca_data.shape)

# Runs models on PCA to see if there is a difference in the results
pca_models = run_features(all_pca_data, [], 'converted_goals', ['log', 'knn', 'gnb', 'randomForest'])

#preds = knn.predict(pca_X)
(pca_y, pca_X) = model_libs._extract_target(all_pca_data, 'converted_goals')

for m in pca_models:
    check_accuracy(m, pca_X)

Cross Validating the SVC model with the PCA data to help prevent overfitting

In [ ]:
pca_svc_model = form_model.build_tuned_model(pca_X, pca_y, 'svc')

In [10]:
prediction_models = form_model.load_models(['knn', 'svc', 'randomForest', 'gnb', 'gmm'])

Success :: Loaded - knn
Success :: Loaded - svc
Success :: Loaded - randomForest
Success :: Loaded - gnb
Success :: Loaded - gmm


In [11]:
upcoming_matches, match_details = predict_matches.get_upcoming_matches()
upcoming_matches.to_csv('upcoming_matches.csv')
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
upcoming_data = predict_matches.predictions(upcoming_matches)
#upcoming_data.to_csv('upcoming_data.csv')
display(upcoming_data.head())

,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,goals_for,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks,goals,points
0,900,21,FC Dallas,23,Real Salt Lake,2016-09-25 01:30:00,28,3,0,4-2-3-1,5,5,-144.000000,-28.777778,-7114.666667,-6425.333333,3773.000000,-472.888889,0,0
1,896,22,Colorado Rapids,28,Vancouver Whitecaps FC,2016-09-24 23:00:00,28,3,0,4-2-3-1,1,3,-63.000000,-44.777778,-7363.555556,-6452.444444,892.888889,2422.333333,0,0
2,900,23,Real Salt Lake,21,FC Dallas,2016-09-25 01:30:00,28,3,1,4-3-3,3,4,-119.444444,-71.111111,-8329.444444,-7900.000000,3021.333333,-1758.333333,0,0
3,894,24,New York City FC,40,Chicago Fire,2016-09-24 00:00:00,28,3,1,4-3-3,6,4,-51.111111,-153.000000,-7900.000000,-8425.666667,-1758.333333,623.000000,0,0
4,895,25,Philadelphia Union,34,Toronto FC,2016-09-24 21:00:00,28,3,0,4-2-3-1,2,5,-102.777778,-75.000000,-9170.333333,-7105.000000,1885.333333,765.333333,0,0


In [12]:
""" Need to format the data the same way we did on the raw data """
formatted_upcoming_data = upcoming_data.drop('points', 1)

formatted_upcoming_data['converted_goals'] = formatted_upcoming_data.apply(lambda row: model_libs.set_group(row['goals']), axis=1)

formatted_upcoming_data = formatted_upcoming_data.drop(ignore_cols + ['current_formation', 'goals', 'converted_goals'], 1)
display(formatted_upcoming_data.head())

,is_home,goals_for,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks
0,0,5,5,-144.000000,-28.777778,-7114.666667,-6425.333333,3773.000000,-472.888889
1,0,1,3,-63.000000,-44.777778,-7363.555556,-6452.444444,892.888889,2422.333333
2,1,3,4,-119.444444,-71.111111,-8329.444444,-7900.000000,3021.333333,-1758.333333
3,1,6,4,-51.111111,-153.000000,-7900.000000,-8425.666667,-1758.333333,623.000000
4,0,2,5,-102.777778,-75.000000,-9170.333333,-7105.000000,1885.333333,765.333333


In [14]:
""" Models we'll use to predict on upcoming matches """
# pca_svc_model, knn_model, random_forest_model

# This is all the X values
formatted_upcoming_data

rf_preds = prediction_models[2].predict(formatted_upcoming_data)
print(rf_preds)
knn_preds = prediction_models[0].predict(formatted_upcoming_data)
print(knn_preds)

svc_preds = prediction_models[1].predict(formatted_upcoming_data)
print(svc_preds)

gmm_preds = prediction_models[3].predict(formatted_upcoming_data)
print(gmm_preds)

gnb_preds = prediction_models[3].predict(formatted_upcoming_data)
print(gnb_preds)

#linear_preds = lin_regr.predict(upcoming_ranked)
#print(linear_preds)


[0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0
 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1
 1 1 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1
 1 1 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]


In [15]:
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones above
upcoming_matches = upcoming_data[columns]

# Add predictions to the end of that DF
results = pd.DataFrame({'KNN': knn_preds, 'RandomForest': rf_preds, 'SVC': svc_preds, 'GNB': gnb_preds, 'GMM': gmm_preds})
upcoming_matches = pd.concat([upcoming_matches, results], axis = 1)
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'KNN', 'RandomForest', 'SVC', 'GNB', 'GMM']
reordered_matches = reordered_matches[columns]
reordered_matches.to_csv('predictions_new_model.csv')
print('Prediction CSV saved')

Prediction CSV saved


In [20]:
actual_data = pd.read_csv('predictions_new_model.csv')
display(actual_data.head())
actual_data = actual_data.drop(actual_data.columns[[0]], axis=1)
actual_data['diff1'] = actual_data.apply(lambda r: model_libs.predictions_diff(r['Actual'], r['GMM']), axis=1)
accuracy = np.divide(actual_data['diff1'].sum(), float(len(actual_data['diff1'])))
print(accuracy)

,Unnamed: 0,scheduled,team_name,opp_name,KNN,RandomForest,SVC,GNB,GMM,Actual
0,0,9/25/16 1:30,FC Dallas,Real Salt Lake,0,0,0,0,0,0
1,2,9/25/16 1:30,Real Salt Lake,FC Dallas,0,1,0,0,0,0
2,1,9/24/16 23:00,Colorado Rapids,Vancouver Whitecaps FC,0,0,0,0,0,1
3,7,9/24/16 23:00,Vancouver Whitecaps FC,Colorado Rapids,0,0,0,0,0,1
4,3,9/24/16 0:00,New York City FC,Chicago Fire,0,0,0,1,1,1


0.591836734694
